In [7]:
%pip install -q langchain_google_genai

In [21]:
from google.colab import auth
auth.authenticate_user()

In [22]:
import langchain_google_genai
import time

In [23]:
def get_embedding_with_retry(text, max_retries=5, initial_delay=1):
    """Gets an embedding with retry logic."""
    for attempt in range(max_retries):
        try:
            llm = langchain_google_genai.GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-exp-03-07")
            embedding = llm.embed_query(text)
            return embedding
        except Exception as e:
            if "Deadline Exceeded" in str(e):
                print(f"Attempt {attempt + 1} failed: Deadline Exceeded. Retrying in {initial_delay} seconds...")
                time.sleep(initial_delay)
                initial_delay *= 2  # Exponential backoff
            else:
                print(f"Attempt {attempt + 1} failed with a different error: {e}")
                raise  # Re-raise the exception if it's not a timeout

    raise Exception("Max retries reached. Embedding failed after multiple attempts.")


In [24]:
# Example usage:
try:
    text_to_embed = "This is a sample text for embedding."
    embedding_vector = get_embedding_with_retry(text_to_embed)
    print("Embedding:", embedding_vector)
except Exception as e:
    print("Error getting embedding:", e)

Attempt 1 failed with a different error: Error embedding content: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.EmbedContent"
}
]
Error getting embedding: Error embedding content: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.EmbedContent"
}
]
